In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [53]:
# logging 
import logging
import logging.config

# allow loading modules from strings
import importlib

# shell utilities for manpiulating files
from shutil import copyfile

# parse configuration files
import configparser

# parse command line arguments
import argparse

# handle file paths in a sane way
from pathlib import Path

# clock 
from datetime import datetime

# loop delay - sleep
import time


######### PyPi libraries
# LMS Query interface
import lmsquery

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

In [28]:
# local libraries
import epdlib

In [4]:
# # this works best as a global variable
# # FIXME - best practice for specifying this?
# logConfig = Path('./logging.cfg')
# logging.config.fileConfig(logConfig.absolute())
# # logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

# logger = logging.getLogger(__name__)

In [5]:
def arg_parse():
    '''parse arguments for this script
    
    Arguments parsed:
        -c: configuration file
    
    Returns:
        :obj:`argpars.ArgumentParser`'''
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
    
    #FIXME add log level with -v -vv -vvv etc.
    
    parser.add_argument('-q', type=str, required=False,
                       help='this is for testing only')
    
    args, unknown = parser.parse_known_args()
    logging.info(f'discarding unknwon commandline arguments: {unknown}')
    
    return args

In [6]:
def read_config(cfgfile=None, default=None):
    '''read `cfgfile` file and optionally create one if it does not exist

    Args:
        cfgfile(str): path to configuration file to be read
        default(str): path to default configuration file that should be 
            used if cfgfile does not exist -- this is useful for creating
            config files on first run
    
    Returns:
        obj:configparser.ConfigParser
    '''
    if not cfgfile:
        return {}
    
    cfgfile = Path(cfgfile).expanduser().resolve()
    default = Path(default).expanduser().resolve() if default else None
    
    # check if the path exists
    logging.debug(f'creating parent directory (if needed): {cfgfile.parent}')
    Path(cfgfile.parent).mkdir(parents=True, exist_ok=True)

    # check if specified file exists, otherwise copy the default (if provided)    
    if not cfgfile.exists() and default:
        logging.debug(f'copying {default} to {cfgfile}')
        try:
            copyfile(default, cfgfile)
        except (FileNotFoundError, PermissionError) as e:
            print(f'failed to copy default file ({default}) to specified file: {e}')
            return False
#     elif not default:
#         raise FileNotFoundError(f'could not open {cfgfile}')
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    return config

In [47]:
def config_2dict(configuration):
    '''convert an argparse object into a dictionary
    
    Args:
        configuration(:obj:`argpars.ArgumentParser`)
        
    Returns:
        `dict`'''
    d = {}
    for section in configuration.sections():
        d[section] = {}
        for opt in configuration.options(section):
            d[section][opt] = configuration.get(section, opt)
    
    return d

In [56]:
def time_now():
    '''return current time in HH:MM format
        e.g. 04:44, 23:16
    Returns:
        `str`'''
    return datetime.now().strftime("%H:%M")

In [57]:
def limit_callback(until):
    '''callback function for RateLimiter; provides logging information'''
    duration = int(round(until - time.time()))
    logging.debug(f'function call rate limited sleeping for {duration} seconds')

In [58]:
def main():
    # constants
    waveshare = 'waveshare_epd.' # note trailing '.' !!
    layouts = 'epdlib.layouts'
       
    # setup logging
    logingConfig = Path('./logging.cfg').resolve()
    logging.config.fileConfig(logingConfig)
    # logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')
    logger = logging.getLogger(__name__)
    
    appShortName = 'slimpi'
    name = 'com.txoof.'
    appLongName = name+appShortName 
    
    # Configuration Variables
    
    # execute the program - use for checking configuration state 
    execute = True
    # name
       
    
    # Default configuration file if none is specified
    builtin_cfg = Path('./slimpi.cfg')
    # standard location for stored user configuration
    user_cfg = Path(f'~/.config/{appLongName}/slimpi.cfg').expanduser()  
    
    # parse command line arguments
    args = arg_parse()
    
    # use a specified configuration file
    if args.c:
        user_cfg = args.c
        
    logging.info(f'using configuration file: {user_cfg}')        
    
    # read configuration file
    # config parser object
    configuration = read_config(cfgfile=user_cfg, default=builtin_cfg)
    execute = True if configuration else False
    # dictionary
    cfg = config_2dict(configuration)
    
    
    # get all the configuration variables 
    try:
        # lms server settings
        lms_server = cfg['lms_server']
        
        # set epd type - Raises KeyError
        epd = importlib.import_module('waveshare_epd.'+cfg['layouts']['display'])
        
        # set layout types - Raises AttirbuteError, KeyError
        layouts = importlib.import_module(layouts)
        playing_layout_format = getattr(layouts, cfg['layouts']['now_playing'])
        stopped_layout_format = getattr(layouts,cfg['layouts']['stopped'])

        # set log level - Raises KeyError
        log_level = cfg['logging']['log_level']
        
    except (KeyError) as e:
        logging.error(f'error locating section/option in configuration file: {e}')
        execute = False
    except AttributeError as e:
        logging.error(f'error accessing attribute: {e}')

    # setup environment
    
    ########## LMS Query object
    try:
        pass
        lms = lmsquery.LMSQuery(host=lms_server['host'], port=lms_server, player_id=lms_server['player_id'])
    except KeyError as e:
        #FIXME add humanfriendly option to find server and specify player id
        logging.warning(f'missing configuration value for lms_server: {e}')
        execute = False
        
    if not lms.lms_servers:
        logging.warning('No lms servers found on local network')
    
    
    ########## Layout objects - for managing display blocks
    music_layout = epdlib.Layout(layout=playing_layout_format)
    stopped_layout = epdlib.Layout(layout=stopped_layout_format)
    stopped_layout.update_contents({'time': time_now(), 'mode': ' '})
    
    ########## Screen objects - for managing writing to screen
    screen = epdlib.Screen()
    screen.epd = epd.EPD()
    screen.initEPD()
    screen.elements=music_layout.blocks.values()
    
    ########## Flow control variables
    refresh = False # refresh the screen 
#     updated = 0 # last time screen was updated
#     last_update = 0 # time of last update

    ########## Now-Playing Values
    nowplaying_id = None
    nowplaying_mode = None
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3, callback=limit_callback)

        
    # if execute was set false anywhere, bail out
    if not execute:
        print('Exiting due to previous errors')
        return cfg
    

    
    
    return cfg

In [59]:
if __name__ == '__main__':
    foo = main()
print(foo)

<ipython-input-5-faaae8993afe>:arg_parse:19:INFO - discarding unknwon commandline arguments: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-7dba19d7-f7b9-4f29-aa58-209cc815b07c.json']
<ipython-input-58-2f7fed9c377a>:main:35:INFO - using configuration file: ./slimpi.cfg
<ipython-input-6-82fb72048f05>:read_config:20:DEBUG - creating parent directory (if needed): /home/pi/src/slimpi_epd
<ipython-input-6-82fb72048f05>:read_config:35:INFO - reading config file /home/pi/src/slimpi_epd/slimpi.cfg
Layout:layout:192:DEBUG - calculating values from layout for resolution (600, 448)
Layout:_calculate_layout:225:DEBUG - ***title***
Layout:_check_keys:108:DEBUG - checking key/values
Layout:_check_keys:113:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:113:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:113:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:232:DEBUG - dimensions: (600, 256)
Layout:_calcul

Layout:_check_keys:113:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:232:DEBUG - dimensions: (600, 392)
Layout:_calculate_layout:261:DEBUG - has explict position
Layout:_calculate_layout:263:DEBUG - abs_coordinates: (0, 0)
Layout:_scalefont:134:DEBUG - calculating font size
Layout:_scalefont:135:DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Ubuntu/Ubuntu-Regular.ttf
Layout:_scalefont:147:DEBUG - target X font dimension 510.0
Layout:_scalefont:148:DEBUG - target Y dimension: 274.4
Layout:_scalefont:162:DEBUG - X target exceeded
Layout:_scalefont:178:DEBUG - test string: W W W ; dimensions for fontsize 146: (513, 137)
Layout:_calculate_layout:225:DEBUG - ***mode***
Layout:_check_keys:108:DEBUG - checking key/values
Layout:_check_keys:113:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:113:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:232:DEBUG - dimensions: (600, 56)
Lay

In [ ]:
myL = importlib.import_module('epdlib.layouts')
dir(myL)

In [ ]:
myL.'threeRow'

In [39]:
import sys
sys.argv.append('-c')
sys.argv.append('./slimpi.cfg')

In [31]:
sys.argv


['/home/pi/.local/share/virtualenvs/slimpi_epd-b1Rf9la8/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/pi/.local/share/jupyter/runtime/kernel-7dba19d7-f7b9-4f29-aa58-209cc815b07c.json',
 '-c',
 '~/slimpi.cfg']